<a href="https://colab.research.google.com/github/VP1606/nlp_c2/blob/main/11372282_CW2_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Hi there!
This is the code template for CW2 task2 of COMP34711 2025/26.

- <span style="color:red; font-size:1em">First of all, please rename the notebook into "{your_student_id}_CW2_task{your_task_number}.ipynb", for example "12345678_CW2_task2.ipynb".</span>

- In this template, we only provide the minimal structure for your coursework.
  
- Please carefully read and organize your code in the template we provided.

## Constants

In [2]:
#Please keep only necessary information in this cell.

#----------------------Please keep all following constants unchanged.----------------------------------------
NUM_ROWS_VALIDATION = 1031 # Number of rows in validation set
NUM_ROWS_TEST = 1053 # Number of rows in test set

#----------------------Please modify the following constants to fit your actual value.-----------------------
STUDENT_ID = '11372282'  # Replace with your actual 8-digits student ID
TRAINING_SET = '/content/drive/MyDrive/NLP_C2/data/CW2_training_dataset.csv' # Replace with the actual path to your training dataset csv file
VALIDATION_SET = '/content/drive/MyDrive/NLP_C2/data/CW2_validation_dataset.csv'  # Replace with the actual path to your validation dataset csv file
VALIDATION_SET_OUTPUT = f'/content/drive/MyDrive/NLP_C2/data/{STUDENT_ID}_CW2_task2_validation_results.csv'  # Replace with the actual path to your validation prediction csv file
TEST_SET_INPUT = '/content/drive/MyDrive/NLP_C2/data/CW2_test_dataset.csv'  # Replace with the actual path to your test prediction csv file

#----------------------Your constants------------------------------------------------
# By adding more constants here, you can help improve the clarity and maintainability of your code and make the reviewing easier for TAs.

## Installations

In [24]:
# Install required packages for the coursework
# Uncomment and run the following lines if needed

# !pip install pandas scikit-learn --quiet
# !pip install -U transformers
# !pip install sentencepiece

## Imports

In [4]:
#Please keep all imports of your code cells in this cell

#---------------------Required imports----------------------
import pandas as pd
import re
import sys
import os.path
import csv
from sklearn.metrics import f1_score
#----------------------Your imports-------------------------
from transformers import pipeline


## Start of your code cells

- The code cells provided below are demo code format for TAs to quickly locate your implementation.

- You have full right to freely add/delete/edit the titles and codes in the following cells.

- Please follow this genre order: "comedy, cult, flashback, historical, revenge, romantic, scifi, violence".

### Data Loading

#### Constants

In [5]:
TITLE_COLUMN_NAME = "title"
PLOT_SYNOPSIS_COLUMN_NAME = "plot_synopsis"
MERGED_PREPROCESSED_COLUMN_NAME = "merged_preprocessed_text"

FILM_CATEGORIES = ["comedy", "cult", "flashback", "historical", "revenge", "romantic", "scifi", "violence"]

#### Load Training & Validation Data

In [6]:
## --- Load Training CSV ---
training_df = pd.read_csv(TRAINING_SET)
validation_df = pd.read_csv(VALIDATION_SET)

In [7]:
training_df.head()

,ID,title,plot_synopsis,comedy,cult,flashback,historical,revenge,romantic,scifi,violence
0,ee7722b2-bc23-400b-9461-4ff91f01f486,Next of Kin,"Truman Gates (Patrick Swayze), raised in Appal...",0,0,0,0,1,0,0,1
1,3b111f7d-0c19-4cb3-84a1-d6dc687c9716,The Survivalist,The film takes place when oil production has c...,0,1,0,0,0,0,0,0
2,3116352f-4b50-43a2-b9be-458c4aa086e5,Superman II,"Before the destruction of Krypton, the crimina...",0,0,0,0,1,0,0,1
3,bbb71d71-1503-4aa6-9129-918b9efc3c3f,The Hunchback of Notre Dame,The gypsy Esmeralda captures the hearts of man...,0,0,0,0,0,1,0,0
4,c12f67ca-5825-43d0-b9a7-61c348915715,Taxi,Taxi portrays director Jafar Panahi as he cour...,0,0,0,0,0,0,0,1


In [8]:
training_df['plot_synopsis'].values.tolist()[0]

'Truman Gates (Patrick Swayze), raised in Appalachia, has migrated to Chicago to become a police officer. Married to Jessie (Helen Hunt), who is pregnant, he seems to have made the transition from hillbilly to respectable lawman. When the local coal mine closes, Truman persuades his younger brother Gerald (Bill Paxton) to look for work in Chicago. But things take a turn for the worse when soon after landing a job as a truck driver, Gerald\'s vehicle is hijacked by mobsters and Gerald is killed by Joey Rosellini (Adam Baldwin), the nephew of mob boss Poppa John Isabella (Andreas Katsulas).\nTruman returns to Kentucky for the funeral. When his surviving brother, Briar Gates (Liam Neeson), insists on a traditional mountain blood feud, Truman urges his family to let the police deal with Gerald\'s murder. Briar finds Truman\'s reluctance to be disgraceful. Determined to deal with the murderers in his own way, Briar travels to Chicago in search of his youngest brother\'s killer.\nMeanwhile, 

In [9]:
validation_df.head()

,ID,title,plot_synopsis,comedy,cult,flashback,historical,revenge,romantic,scifi,violence
0,df7e125e-2d59-40e4-a126-9397e3a0ef21,Point Blank,Walker works with his friend Mal Reese to stea...,0,1,1,0,1,0,0,1
1,4879a106-9abb-492e-b509-048bcc536f79,Luther,"The film begins during a thunderstorm in 1505,...",0,0,0,0,0,1,0,1
2,33dc5da1-7891-4ce2-8ec3-a8b39d2b4065,Far from the Madding Crowd,Gabriel Oak is a young shepherd. With the savi...,0,0,0,0,0,1,0,0
3,a770699c-939c-4cd5-aa1c-d1f60acc22d6,Outrage,"Three disparate travelers, a disillusioned pre...",0,0,0,0,1,0,0,0
4,da892974-ba16-4f34-8816-ad6535a83590,Breaking Point,"Harry Morgan (John Garfield), is a sport-fishi...",0,0,0,0,0,0,0,1


#### Merge Title & Synopsis

Produce a merged representation, using RoBERTa's seperator tag as follows:

```
<title> TITLE </title> <synopsis> SYNOPSIS </synopsis>

In [10]:
## -- Extract Title & Synopsis --
training_title_list = training_df[TITLE_COLUMN_NAME].tolist()
training_synopsis_list = training_df[PLOT_SYNOPSIS_COLUMN_NAME].tolist()

validation_title_list = validation_df[TITLE_COLUMN_NAME].tolist()
validation_synopsis_list = validation_df[PLOT_SYNOPSIS_COLUMN_NAME].tolist()

In [11]:
## -- Merge using the seperator tag --
training_combined_input = [
    f"<title> {title} </title> <synopsis> {synopsis} </synopsis>"
    for title, synopsis in zip(training_title_list, training_synopsis_list)
]

validation_combined_input = [
    f"<title> {title} </title> <synopsis> {synopsis} </synopsis>"
    for title, synopsis in zip(validation_title_list, validation_synopsis_list)
]

#### Produce Class Labels Array

In [12]:
training_class_labels = training_df[FILM_CATEGORIES].values
validation_class_labels = validation_df[FILM_CATEGORIES].values

### Tokenization

Tokenize using Roberta's Tokenzier. Pre-processing should not be needed, since transformers are robust and trained on real-world text.

#### Intialize Tokenizer

In [34]:
MODEL_NAME = "microsoft/deberta-v3-base"

In [35]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


#### Tokenize Combined Sequenced Representation

In [36]:
TOKEN_MAX_LENGTH = 512

In [37]:
training_input_encoded = tokenizer(
    training_combined_input,
    padding=True,
    truncation=True,
    max_length=TOKEN_MAX_LENGTH
)

In [38]:
validation_input_encoded = tokenizer(
    validation_combined_input,
    padding=True,
    truncation=True,
    max_length=TOKEN_MAX_LENGTH
)

### Define & Build Dataset (Training)

#### Define Dataset Class

In [52]:
import torch

class MovieDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels # It's already an array, so just store it

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Create float tensor for labels
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

#### Build Training & Validation Dataset

In [53]:
training_dataset = MovieDataset(training_input_encoded, training_class_labels)
validation_dataset = MovieDataset(validation_input_encoded, validation_class_labels)

### RoBERTa Model

#### Constants

In [102]:
NUM_CLASSES = 8
# MODEL_NAME = "FacebookAI/roberta-base"
MODEL_NAME = "microsoft/deberta-v3-base"

NUM_EPOCHS = 15

TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 4

# Physical (4) * Accumulation (2) = Effective (8)
GRAD_ACCUMULATION_STEPS = 2

# This uses 16-bit math for some operations, cutting memory usage by ~50%
USE_FP16 = True

# 4. Enable Gradient Checkpointing (The "Silver Bullet" for OOM)
# This forgets intermediate states and re-calculates them when needed.
# It slows training slightly (~20%) but massively reduces memory usage.
USE_GRADIENT_CHECKPOINTING = True

LOGGING_STEPS = 50

EPOCH_EARLY_STOPPING_PATIENCE = 3

#### Class Weights & Loss Metric

In [103]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import torch

# --- 1. Calculate Class Weights Dynamically ---
# This looks at your actual y_train data to find the imbalance.
# Formula: (Total_Samples / Positive_Samples)
# This tells the model: "Pay X times more attention to this class"
# shape of y_train: (n_samples, 8)
positive_counts = np.sum(training_class_labels, axis=0)
total_samples = len(training_class_labels)

# Avoid division by zero with np.maximum
class_weights = total_samples / np.maximum(positive_counts, 1)

print("Calculated Class Weights:", class_weights)
# Convert to tensor for later use
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# --- 2. Define Compute Metrics ---
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Apply threshold: positive if logit > 0
    predictions = (logits > 0).astype(int)

    # Calculate weighted F1 (The Coursework Metric)
    f1 = f1_score(labels, predictions, average='weighted', zero_division=0)
    acc = accuracy_score(labels, predictions)

    return {'f1_weighted': f1, 'accuracy': acc}

Calculated Class Weights: [ 5.79430894  3.95724597  3.57243108 37.31413613  4.2422619   3.57601606
 34.42995169  2.35059367]


#### Trainer

In [104]:
from transformers import Trainer
import torch.nn as nn

class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        weights = class_weights_tensor.to(logits.device)

        # Weighted Loss
        loss_fct = nn.BCEWithLogitsLoss(pos_weight=weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

#### Model Definition

In [105]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, EarlyStoppingCallback

# 1. Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=8,
    problem_type="multi_label_classification"
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [106]:
# 2. Define Arguments (T4 Optimized)
training_args = TrainingArguments(
    output_dir='./results_task3',
    num_train_epochs=NUM_EPOCHS,

    # Batch Size & Accumulation (Effective Batch Size = 8)
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=4,

    # Optimization Strategy
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=2e-5, # DeBERTa prefers lower LRs (1e-5 to 3e-5)
    weight_decay=0.01,

    # Memory Saving (Crucial for T4)
    fp16=True,
    # gradient_checkpointing=True,

    # Best Model Selection
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    report_to="none"
)


In [125]:
# 1. Create tiny subsets (just 50 items)
train_subset = torch.utils.data.Subset(training_dataset, range(50))
# val_subset = torch.utils.data.Subset(validation_dataset, range(50))

In [126]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/tmp/ipython-input-1232779750.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedLossTrainer(


In [127]:
# 4. Train
print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss,F1 Weighted,Accuracy
1,No log,1.231937,0.281214,0.004850
2,No log,1.248676,0.281159,0.003880
3,No log,1.265180,0.281159,0.003880
4,No log,1.302533,0.281159,0.003880


TrainOutput(global_step=28, training_loss=0.9694446836199079, metrics={'train_runtime': 126.8927, 'train_samples_per_second': 5.911, 'train_steps_per_second': 0.827, 'total_flos': 52625989632000.0, 'train_loss': 0.9694446836199079, 'epoch': 4.0})

In [128]:
# Check the best checkpoint path
print(f"Best model loaded from: {trainer.state.best_model_checkpoint}")

# Check the score of that model
print(f"Best F1 Weighted score: {trainer.state.best_metric}")

Best model loaded from: ./results_task3/checkpoint-7
Best F1 Weighted score: 0.281213714718357


### Calculate Predictions (Validation)

In [129]:
import torch
validation_predictions = trainer.predict(validation_dataset)
validation_probabilities = torch.sigmoid(torch.tensor(validation_predictions.predictions)).numpy()

### Optimize Thresholds

As stated in the manual, all optimizing will be performed using the **validation dataset**.

Perform a search over the range (0.01, 0.99) using a step of 0.01 to evaluate which threshold is the best for each class.

In [130]:
import numpy as np
CLASS_THRESHOLDS = np.array([0.2] * NUM_CLASSES)

In [131]:
def optimize_thresholds_per_class(probabilities, targets, step_size=0.01, num_classes=NUM_CLASSES):
  optimal_thresholds = []
  optimal_f1_scores = []

  for class_index in range(num_classes):
    best_f1_score, best_threshold = 0.0, 0.5

    for threshold_trial in np.arange(0.01, 0.99, step_size):
      predictions = (probabilities[:, class_index] > threshold_trial).astype(int)
      f1_score_trial = f1_score(targets[:, class_index], predictions, zero_division=0)

      if f1_score_trial > best_f1_score:
        best_f1_score = f1_score_trial
        best_threshold = threshold_trial

    optimal_thresholds.append(best_threshold)
    optimal_f1_scores.append(best_f1_score)

  return optimal_thresholds, optimal_f1_scores

In [132]:
optimal_thresholds_calculated, optimal_f1_scores_calculated = optimize_thresholds_per_class(
    probabilities=validation_probabilities,
    targets=validation_class_labels,
    step_size=0.01
)

In [133]:
print(optimal_thresholds_calculated)

[np.float64(0.42000000000000004), np.float64(0.45), np.float64(0.37), np.float64(0.49), np.float64(0.01), np.float64(0.5800000000000001), np.float64(0.34), np.float64(0.52)]


In [134]:
print(optimal_f1_scores_calculated)

[0.3114754098360656, 0.3926282051282051, 0.4429327286470144, 0.07692307692307693, 0.3827450980392157, 0.4397270659590599, 0.07858546168958742, 0.6099865047233468]


### Submit Solution

In [135]:
predictions = (validation_probabilities > optimal_thresholds_calculated).astype(int)

In [136]:
id_list = validation_df['ID'].values.tolist()

In [137]:
submission = pd.DataFrame({"ID": id_list})
for i, genre in enumerate(FILM_CATEGORIES):
    submission[genre] = predictions[:, i]

In [138]:
submission.head()

,ID,comedy,cult,flashback,historical,revenge,romantic,scifi,violence
0,df7e125e-2d59-40e4-a126-9397e3a0ef21,1,1,1,0,1,1,0,1
1,4879a106-9abb-492e-b509-048bcc536f79,1,1,1,0,1,1,0,1
2,33dc5da1-7891-4ce2-8ec3-a8b39d2b4065,1,1,1,0,1,1,1,1
3,a770699c-939c-4cd5-aa1c-d1f60acc22d6,1,1,1,0,1,1,1,1
4,da892974-ba16-4f34-8816-ad6535a83590,1,1,1,0,1,1,0,1


In [139]:
submission.to_csv(VALIDATION_SET_OUTPUT, index=False, header=False)

## End of your code cells

### Evaluation scripts

In [140]:
def read_data(submission_file_path, gold_standard_file_path):
    """
    Read submission and gold standard files.
    Extract student ID from filename.
    """
    # Try to find student ID from the filename (looks for 8 digit numbers)
    id_regex = r'\d{8}'

    user_id = re.findall(id_regex, submission_file_path)
    print("Found your ID: ", user_id)
    if user_id:
        user_id = user_id[0]
    else:
        user_id = 'Unknown'

    # Load submission CSV
    print(f"\nLoading submission file: {submission_file_path}")
    submission_df = pd.read_csv(submission_file_path, sep=',', header=None,
                                quoting=csv.QUOTE_NONE, encoding='utf-8')

    # Load gold standard CSV
    print(f"Loading gold standard file: {gold_standard_file_path}")
    gold_standard_df = pd.read_csv(gold_standard_file_path, header=None)

    # Remove columns 1 and 2 (keep only ID and labels)
    gold_standard_df = gold_standard_df.drop([1, 2], axis=1)
    # Skip header row
    gold_standard_df = gold_standard_df.iloc[1:]

    return submission_df, gold_standard_df, user_id


def match_and_prepare_data(submission_df, gold_standard_df, user_id):
    """
    Match submission rows with gold standard rows by ID.
    Prepare data for evaluation.
    """
    gold_standard_labels = []
    submission_labels = []
    missed_rows = []
    submission_df_copy = submission_df.copy()

    print(f"\nMatching submission with gold standard...")
    print(f"Gold standard rows: {len(gold_standard_df)}")
    print(f"Submission rows: {len(submission_df_copy)}")

    # Match each gold standard row with submission
    for index, row in gold_standard_df.iterrows():
        row = row.reset_index(drop=True)
        row_found = False
        row_id = row[0]

        # Extract gold standard labels
        row_labels = [int(row[i]) for i in range(1, len(row))]
        gold_standard_labels.append(row_labels)

        # Find corresponding submission row
        for sub_index, submission_row in submission_df_copy.iterrows():
            if submission_row[0].strip() == row_id.strip():
                try:
                    # Extract submission labels
                    submission_row_labels = [int(submission_row[i]) for i in range(1, len(submission_row))]
                except:
                    # Handle malformed labels (take first character if multi-digit)
                    submission_row_labels = [int(str(submission_row[i])[0]) for i in range(1, len(submission_row))]

                submission_labels.append(submission_row_labels)
                row_found = True
                submission_df_copy.drop(sub_index, inplace=True)
                break

        if not row_found:
            # If row is missing, add inverse labels (worst possible prediction)
            missed_rows.append(row_id)
            submission_labels.append([0 if label == 1 else 1 for label in row_labels])

    return gold_standard_labels, submission_labels, missed_rows


def evaluate_submission(gold_standard_labels, submission_labels):
    """
    Calculate weighted F1 score.
    """
    print(f"\nCalculating weighted F1 score...")

    # Calculate weighted F1 score (accounts for class imbalance)
    f1_weighted = f1_score(gold_standard_labels, submission_labels, average='weighted')

    return f1_weighted


def print_results(user_id, f1_weighted, missed_rows):
    """
    Print evaluation results to screen.
    """
    print("\n" + "="*70)
    print("YOUR SUBMISSION EVALUATION REPORT")
    print("="*70)

    # Alert if ID not found in filename
    if user_id == 'Unknown':
        print('WARNING: ID not found in filename!')
        print('   Please ensure your filename contains your 8-digit student ID.')
        print()

    print(f"Your ID: {user_id}")
    print()

    # Display F1 score with visual indicator
    print("EVALUATION RESULTS:")
    print(f"   Weighted F1 Score: {f1_weighted:.4f}")
    print()

    # Report missing rows
    if missed_rows:
        print(f"MISSING DATA ({len(missed_rows)} rows not found):")
        print("-" * 70)
        for i, row in enumerate(missed_rows[:10], 1):  # Show first 10
            print(f"    {i}. Row ID: {row}")
        if len(missed_rows) > 10:
            print(f"    ... and {len(missed_rows) - 10} more missing rows")
        print()
        print("TIP: Make sure your submission includes all required rows.")
        print("        Missing rows are penalized with worst possible predictions.")
    else:
        print("DATA COMPLETENESS: All expected rows found in your submission!")

    print()
    print("="*70)
    print()


def evaluate(submission_path, gold_standard_path):
    """
    Main function to run the submission evaluation script.
    """

    submission_file = submission_path
    gold_standard_file = gold_standard_path

    # Check if files exist
    if not os.path.exists(submission_file):
        print(f"Error: Your submission file '{submission_file}' not found!")
        print("Make sure the file path is correct and the file exists.")
        sys.exit(1)

    if not os.path.exists(gold_standard_file):
        print(f"Error: Gold standard file '{gold_standard_file}' not found!")
        print("Make sure you have the correct gold standard file.")
        sys.exit(1)

    try:
        # Step 1: Read data
        submission_df, gold_standard_df, user_id = read_data(submission_file, gold_standard_file)

        # Step 2: Match and prepare data
        gold_standard_labels, submission_labels, missed_rows = match_and_prepare_data(
            submission_df, gold_standard_df, user_id
        )

        # Step 3: Evaluate
        f1_weighted = evaluate_submission(gold_standard_labels, submission_labels)

        # Step 4: Print results
        print_results(user_id, f1_weighted, missed_rows)

    except Exception as e:
        print(f"Error during evaluation: {str(e)}")
        print("Please check that your files are in the correct CSV format.")
        print("Each row should contain: ID, label1, label2, label3, ...")
        import traceback
        traceback.print_exc()
        sys.exit(1)

### Evaluate the model on the validation dataset

In [141]:
# Please run the evaluation scripts cell above before running the mark_and_record

# Please make sure that output format is like following (no header row, no tilte and plot columns):
# 94834c61-0e30-4799-9998-6f74f6sbb204	0	1	0	0	1	0	0	0
# 559sdd28-b6a2-4662-ab55-a6678as26a56	0	0	0	0	0	0	1	0
# b71y3317-04cd-42f5-a380-d21dfasdbd36	0	0	0	0	1	0	0	0

evaluation_results = evaluate(VALIDATION_SET_OUTPUT, VALIDATION_SET)

Found your ID:  ['11372282']

Loading submission file: /content/drive/MyDrive/NLP_C2/data/11372282_CW2_task2_validation_results.csv
Loading gold standard file: /content/drive/MyDrive/NLP_C2/data/CW2_validation_dataset.csv

Matching submission with gold standard...
Gold standard rows: 1031
Submission rows: 1031

Calculating weighted F1 score...

YOUR SUBMISSION EVALUATION REPORT
Your ID: 11372282

EVALUATION RESULTS:
   Weighted F1 Score: 0.4459

DATA COMPLETENESS: All expected rows found in your submission!




### Save predictions to formatted file.

In [142]:
# Now please modify the code to format your output csv file.

# Please make sure that output format is like following (no header row, no title and plot columns):
# 94834c61-0e30-4799-9998-6f74f6sbb204	0	1	0	0	1	0	0	0
# 559sdd28-b6a2-4662-ab55-a6678as26a56	0	0	0	0	0	0	1	0
# b71y3317-04cd-42f5-a380-d21dfasdbd36	0	0	0	0	1	0	0	0

output_df = None  # Replace with your actual DataFrame or output
# For example, if you have a DataFrame named 'output_df', you can save it
assert isinstance(output_df, pd.DataFrame)
assert len(output_df) == NUM_ROWS_TEST, "Output length is not aligned with the testdata.csv."
assert len(output_df.columns) == 9, "Please make sure to follow the format above and keep only IDs and 8 columns of prediction."
output_df.to_csv(f'./{STUDENT_ID}_CW2_task2_results.csv', index=False, header=False)

AssertionError: 